## Procesamiento de datos de trafico vehicular

Esta libreta contiene los procedimientos realizados para procesar datos de conteos vehiculares de cámaras de la ciudad de montevideo, cedidos por la Intendencia de Montevideo (IM) y por el Ministerio de Transporte y Obras Públicas (MTOP) buscando generar un dataset de entrenamiento de redes neuronales. 

Este trabajo se enmarca en la tesis de Maestría en Ciencia de Datos de la UTEC. 

Esta libreta no fue pensada para ser ejecutada por un tercero, sino para que el equipo y evaluadores revisaran el proceso de manejo de datos

In [5]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from os import listdir
import glob 
import sqlite3
import json

In [6]:
path_test = 'dataset\test' 
path_all =  'dataset'

Lista de los nombres de las columnas

'ï»¿cod_detector' 

'id_carril' 

'fecha' 

'hora' 

'dsc_avenida'

'dsc_int_anterior'

'dsc_int_siguiente'

'latitud'

'longitud'       

'volumen'

'volumen_hora'

'cod_detector'       

#### Número de registros por archivo. 

Un desafío que existió fue la falta de consistencia en los formatos de los archivos en los caracteres separadores. 

In [11]:
def get_rows(path=path):
    """Create a dictionary of camaras positions"""
    rows = []
    cols = ['dsc_avenida','dsc_int_anterior','dsc_int_siguiente','latitud','longitud']
    all_files = glob.glob(path + "/*.csv")
    counter = 0 
    
    faltan = ([r'C:\Python\Master Datascience\Tesis\dataset\autoscope_05_2020.csv',
           r'C:\Python\Master Datascience\Tesis\dataset\autoscope_06_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_07_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_08_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_09_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_10_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_11_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_12_2020.csv'
              ])
    chunksize = 10 ** 5
    for filename in all_files:
        row_chunk = []
        if filename in faltan:
            sep=";"
        else:
            sep = ","
        for chunk in pd.read_csv(filename ,header=0 ,encoding="ISO-8859-1",sep=sep,chunksize=chunksize, error_bad_lines=False):
             row_chunk.append(chunk.shape[0])
        rows.append(sum(row_chunk))
        counter += 1
        del chunk
        print("Faltan {}".format(len(all_files)-counter))
    print("Se registran un total de {} registros de camaras, con un maximo y minimo de {} y {} por archivo, siendo un promedio de {}".
         format(sum(rows),max(rows), min(rows),sum(rows)/counter))
    

ValueError: max() arg is an empty sequence

#### Cámaras que cambian de posición

También se detectaron que la ID de la camara no era fija en una posición de la ciudad, sino que variaba con respecto al tiempo

In [8]:
def create_camara_position_relation(path=r'C:\Python\Master Datascience\Tesis\dataset'):
    """Create a dictionary of camaras positions"""
    
    with open('id_calles_norm.json') as d:
        id_calles_norm = json.load(d)
    
    all_files = glob.glob(path + "/*.csv")
    counter = 0 
    result = pd.DataFrame()
    faltan = ([r'C:\Python\Master Datascience\Tesis\dataset\autoscope_05_2020.csv',
           r'C:\Python\Master Datascience\Tesis\dataset\autoscope_06_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_07_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_08_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_09_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_10_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_11_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_12_2020.csv'
              ])
    chunksize = 10 ** 6
    
    for filename in all_files:
        if filename in faltan:
            sep=";"
        else:
            sep = ","
        try:
            for chunk in pd.read_csv(filename ,header=0 ,encoding="ISO-8859-1",sep=sep, chunksize=chunksize, error_bad_lines=False):
                chunk = fix_street_labels(chunk)
                chunk.rename(columns={chunk.columns[0]:'id_camara'},inplace=True)
                chunk['mix'] = chunk.dsc_avenida + '/' + chunk.dsc_int_anterior + '/' +  chunk.dsc_int_siguiente
                chunk['id_corregido'] = chunk.mix.map(id_calles_norm)
                dif = chunk[chunk['id_camara']!=chunk['id_corregido']].drop_duplicates(subset=['id_camara','id_corregido'], keep='first')[['id_camara','id_corregido']]
                result = result.append(dif)
            result = result.drop_duplicates(subset=['id_camara','id_corregido'], keep='first')
            counter += 1
            print("Faltan {}".format(len(all_files)-counter))
            print("Print van {} posiciones con repeticion".format(result.shape[0]))
        except Exception as e:
            #df = pd.read_csv(filename,sep=';')
            #del df
            print(e)
        
    return result
dif = create_camara_position_relation()

In [67]:
dif.to_excel("Relacion camaras con id corregido.xlsx",index=False)

In [10]:
# Carga las calles normalizadas en un json

with open('id_calles_norm.json') as d:
    id_calles_norm = json.load(d)

In [11]:
id_calles_norm

{'AV CARLOS MARIA RAMIREZ/Camambu/Acceso Ta-Ta': 1,
 'AV GRAL FLORES/DOMINGO ARAMBURU/Blandengues': 2,
 'RBLA REPUBLICA DEL PERU/JOSE MARIA MONTERO/BV ESPAÑA': 3,
 'AV SARMIENTO/DR JOAQUIN REQUENA/RBLA WILSON': 4,
 'AV DANIEL FERNANDEZ CRESPO/La Paz/Miguelete': 5,
 'DR EVARISTO CIGANDA/AV MILLAN/AV AGRACIADA': 6,
 'Miguelete/Justicia/BV GRAL ARTIGAS': 7,
 'AV MILLAN/Americo Vespucio/BV GRAL ARTIGAS': 8,
 'Camambu/RUTA NAL 1/AV CARLOS MARIA RAMIREZ': 9,
 'AV GRAL GARIBALDI/Monte Caseros/BV GRAL ARTIGAS': 10,
 'Ejido/Canelones/Soriano': 11,
 'AV MCAL FRANCISCO SOLANO LOPEZ/AV GRAL RIVERA/RBLA REPUBLICA DE CHILE': 12,
 'RBLA REPUBLICA DE CHILE/Arrascaeta/Hipolito Yrigoyen': 13,
 'Colonia/Arenal Grande/AV DANIEL FERNANDEZ CRESPO': 14,
 'Magallanes/Mercedes/Colonia': 15,
 'AV GRAL SAN MARTIN/Dr Magested/BV GRAL ARTIGAS': 16,
 'AV GRAL SAN MARTIN/Martin Garcia/AV AGRACIADA': 17,
 'San Jose/Yaguaron/Ejido': 18,
 'AV 8 DE OCTUBRE/Avelino Miranda/PTE BERRO': 19,
 'San Jose/Andes/Convencion': 20

##### Procesamiento de la camara del MTOP en 'RUTA NAL INTERCONEXION RUTA 1 Y 5/AV DR SANTIN CARLOS ROSSI/BV GRAL ARTIGAS': 276

In [5]:
def create_camara_dict(path=r'C:\Python\Master Datascience\Tesis\dataset'):
    """Create a dictionary of camaras positions"""
    result_list = []
    cols = ['dsc_avenida','dsc_int_anterior','dsc_int_siguiente','latitud','longitud']
    all_files = glob.glob(path + "/*.csv")
    counter = 0 
    for filename in all_files:
        try:
            df = pd.read_csv(filename, index_col=None, header=0, encoding="ISO-8859-1",usecols=cols)
            df = fix_street_labels(df)
            df['mix'] = df.dsc_avenida + '/' + df.dsc_int_anterior + '/' +  df.dsc_int_siguiente
            partial_list = df.mix.drop_duplicates(keep='first').to_list()
            [result_list.append(x) for x in partial_list if x not in result_list]
        except Exception as e:
            print(e)
        print("VAN ",filename)
        counter += 1
    del df
    chunksize = 10 ** 5
    faltan = ([r'C:\Python\Master Datascience\Tesis\dataset\autoscope_05_2020.csv',
           r'C:\Python\Master Datascience\Tesis\dataset\autoscope_06_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_07_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_08_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_09_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_10_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_11_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_12_2020.csv'
              ])
    for filename in faltan:
        for chunk in pd.read_csv(filename ,header=0 ,encoding="ISO-8859-1",sep=";",chunksize=chunksize, error_bad_lines=False):
            #chunk = chunk.rename(columns={chunk.columns[0]:'id_camara','volume':'volumen'})
            chunk.drop_duplicates(keep='first',inplace=True)
            chunk = fix_street_labels(chunk)
            chunk['mix']= chunk.dsc_avenida + '/' + chunk.dsc_int_anterior + '/' +  chunk.dsc_int_siguiente
            partial_list = chunk.mix.drop_duplicates(keep='first').to_list()
            [result_list .append(x) for x in partial_list if x not in result_list]
        print("VAN ",counter)
        counter += 1
    return result_list
lista = create_camara_dict()

index = 1
dicti = {}
for mix in lista:
    dicti[mix] = index
    index += 1
dicti['RUTA NAL INTERCONEXION RUTA 1 Y 5/AV DR SANTIN CARLOS ROSSI/BV GRAL ARTIGAS']= index
with open('id_calles_norm.json', "w+") as d: 
    json.dump(lista, d)
print("Archivo guardado con exito")

name 'fix_street_labels' is not defined
VAN  C:\Python\Master Datascience\Tesis\dataset\autoscope_01_2020.csv
name 'fix_street_labels' is not defined
VAN  C:\Python\Master Datascience\Tesis\dataset\autoscope_02_2020.csv
Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.
VAN  C:\Python\Master Datascience\Tesis\dataset\autoscope_03_2020.csv
Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.
VAN  C:\Python\Master Datascience\Tesis\dataset\autoscope_04_2020.csv
Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.
VAN  C:\Python\Master Datascience\Tesis\dataset\Autoscope_05_2019.csv
Usecols do not match columns, columns expected but not found: ['longitud', 'dsc_int_siguiente', 'dsc_avenida', 'dsc_int_anterior', 'latitud']
VAN  C:\Python\Master Datascience\Tesis\dataset\autoscope_05_2020.csv
name 'fix_street_labels' is not defined
VAN  C:\Python\Master Datascience\Tesis\da

NameError: name 'fix_street_labels' is not defined

In [25]:
index = 1
dicti = {}
for mix in lista:
    dicti[mix] = index
    index += 1 #Suma la cámara del MTOP al final del indexado.
dicti['RUTA NAL INTERCONEXION RUTA 1 Y 5/AV DR SANTIN CARLOS ROSSI/BV GRAL ARTIGAS']= index
with open('id_calles_norm.json', "w+") as d: 
    json.dump(dicti, d)
print('Listo')

Listo


In [ ]:
def max_vol(cols='all',path=r'C:\Python\Master Datascience\Tesis\dataset',remove_duplicate=False):
    """Encuentra dentro de los diferentes dataset en cual aparece el maximo volumen de autos por hora 
    (Usando en fase exploratoria)"""
    all_files = glob.glob(path + "/*.csv")
    max_volumen = 0
    final = pd.Series()
    for filename in all_files:
        #print(i)
        df = pd.read_csv(filename, index_col=None, header=0, encoding="ISO-8859-1")
        if df.shape[1]==1:
            del df
            df = pd.read_csv(filename, index_col=None, header=0, sep=";",encoding="ISO-8859-1")
        try:
            max_volumen_df = df['volumen_hora'].max()
            #print(filename,max_volumen_df)
            if max_volumen_df >= max_volumen:
                final = df[df['volumen_hora']==df['volumen_hora'].max()]
                max_volumen = max_volumen_df
        except Exception as e:
            print(e, filename)
            print(df.columns)
        del df
    return final

In [12]:
def select_camara(camara,cols=None,path=r'C:\Python\Master Datascience\Tesis\dataset',remove_duplicate=False):
    ""S"Selecciona todos los datos de la camara en los csv"""
    all_files = glob.glob(path + "/*.csv")
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, encoding="ISO-8859-1")
        if df.shape[1]==1:
            del df
            df = pd.read_csv(filename, index_col=None, header=0, sep=";",encoding="ISO-8859-1",usecols=cols)
        try:
            df = df[df['cod_detector']==camara]
        except Exception as e:
            print(e, df.head)
        li.append(df)
        del df
    return pd.concat(li, axis=0, ignore_index=True)

In [14]:
def open_frames(cols=None,path=r'C:\Python\Master Datascience\Tesis\dataset\test',remove_duplicate=False):
    """Open data csvs and returns specific dataframes"""
    all_files = glob.glob(path + "/*.csv")
    li = []
    #opened_files = 0
    for filename in all_files:
        #print(i)
        df = pd.read_csv(filename, index_col=None, header=0, encoding="ISO-8859-1",usecols=cols)
        print(df.columns)
        try:
            if remove_duplicate: # Removes duplicate values if specificated
                df.drop_duplicates(subset=cols,keep='first',inplace=True)
            li.append(df)
        except:
            print(df.columns)
    return pd.concat(li, axis=0, ignore_index=True)

Otro desafío que encontramos fue que era necesario normalizar los nombres de las calles 

In [14]:
# Arreglo tipos presentes en las calles al correspondiente en la base de esquinas
def fix_street_labels(df):
    """Arregla los errores encontrados en el nombre de las calles"""
    for column in ['dsc_avenida','dsc_int_anterior','dsc_int_siguiente']:
        df[column] = df[column].replace('Comercio', 'AV MCAL FRANCISCO SOLANO LOPEZ')
        df[column] = df[column].replace('21 de setiembre', '21 de Setiembre')
        df[column] = df[column].str.replace('Bv Batlle y Ordonez','BV JOSE BATLLE Y ORDOÑEZ')
        df[column] = df[column].str.replace('Battle y Ordonez','BV JOSE BATLLE Y ORDOÑEZ')
        df[column] = df[column].str.replace('Solano LÃ³pez', 'Solano Lopez')
        df[column] = df[column].str.replace('Batlle y Ordonez','BV JOSE BATLLE Y ORDOÑEZ')
        df[column] = df[column].replace('Gral Flores', 'AV GRAL FLORES')
        df[column] = df[column].replace('Sarmiento', 'AV SARMIENTO')
        df[column] = df[column].replace('Soca', 'AV DR FRANCISCO SOCA')
        df[column] = df[column].replace('Solano López', 'AV MCAL FRANCISCO SOLANO LOPEZ')
        df[column] = df[column].replace('Corrales', 'CNO CORRALES')
        df[column] = df[column].replace('M Herrera y Obes', 'BV MANUEL HERRERA Y OBES')
        df[column] = df[column].replace('Herrera y Obes', 'JULIO HERRERA Y OBES')
        df[column] = df[column].replace('8 de octubre', 'AV 8 DE OCTUBRE')
        df[column] = df[column].replace('Albo', 'AV DR MANUEL ALBO')
        df[column] = df[column].replace('Victor Haedo', 'EDUARDO VICTOR HAEDO')
        df[column] = df[column].replace('Aramburu', 'DOMINGO ARAMBURU')
        df[column] = df[column].replace('Rodo', 'JOSE ENRIQUE RODO')
        df[column] = df[column].replace('Morquio', 'AV DR LUIS MORQUIO')
        df[column] = df[column].replace('Varela', 'AV JOSE PEDRO VARELA')
        df[column] = df[column].replace('Paullier', 'JUAN PAULLIER')
        df[column] = df[column].replace('Bv Espana', 'BV ESPAÑA')
        df[column] = df[column].replace('E V Haedo', 'EDUARDO VICTOR HAEDO')
        df[column] = df[column].replace('Raissignier', 'JUAN J RAISSIGNIER')
        df[column] = df[column].replace('Zum Felde', 'ALBERTO ZUM FELDE')
        df[column] = df[column].replace('Piccioli', 'GERONIMO PICCIOLI')
        df[column] = df[column].replace('Amezaga', 'DR JUAN J DE AMEZAGA')
        df[column] = df[column].replace('Libertador', 'AV LIB BRIG GRAL LAVALLEJA')
        df[column] = df[column].replace('Bolivia', 'AV BOLIVIA')
        df[column] = df[column].replace('Ellauri', 'JOSE ELLAURI')
        df[column] = df[column].replace('Larranaga', 'AV DAMASO ANTONIO LARRAÑAGA')
        df[column] = df[column].replace('Ponce', 'AV ING LUIS P PONCE')
        df[column] = df[column].replace('Suarez', 'AV JOAQUIN SUAREZ')
        df[column] = df[column].replace('Urdaneta', 'GRAL FRANCISCO URDANETA')
        df[column] = df[column].replace('Beloni', 'AV JOSE BELLONI')
        df[column] = df[column].replace('Requena', 'DR JOAQUIN REQUENA')
        df[column] = df[column].replace('Lezica', 'AV LEZICA')
        df[column] = df[column].replace('Burgues', 'AV BURGUES')
        df[column] = df[column].replace('Garzon', 'AV GRAL EUGENIO GARZON')
        df[column] = df[column].replace('Rivera', 'AV GRAL RIVERA')
        df[column] = df[column].replace('Saravia', 'BV APARICIO SARAVIA')
        df[column] = df[column].replace('Uruguay', 'AV URUGUAY')
        df[column] = df[column].replace('Urdaneta', 'GRAL FRANCISCO URDANETA')
        df[column] = df[column].replace('Zufriategui', 'PABLO ZUFRIATEGUI')
        df[column] = df[column].replace('Belloni', 'AV JOSE BELLONI')
        df[column] = df[column].replace('Ricaldoni', 'AV DR AMERICO RICALDONI')
        df[column] = df[column].replace('Centenario', 'AV CENTENARIO')
        df[column] = df[column].replace('Agraciada', 'AV AGRACIADA')
        df[column] = df[column].replace('Luis Batlle Berres', 'AV LUIS BATLLE BERRES')
        df[column] = df[column].replace('San Martin', 'AV GRAL SAN MARTIN')
        df[column] = df[column].replace('8 de Octubre', 'AV 8 DE OCTUBRE')
        df[column] = df[column].replace('Francisco Urdaneta', 'GRAL FRANCISCO URDANETA')
        df[column] = df[column].replace('Daniel Munoz', 'DANIEL MUÑOZ')
        df[column] = df[column].replace('Millan', 'AV MILLAN')
        df[column] = df[column].replace('Garibaldi', 'AV GRAL GARIBALDI')
        df[column] = df[column].replace('Solano Lopez', 'AV MCAL FRANCISCO SOLANO LOPEZ')
        df[column] = df[column].replace('Fernandez Crespo', 'AV DANIEL FERNANDEZ CRESPO')
        df[column] = df[column].replace('Bv Artigas', 'BV GRAL ARTIGAS')
        df[column] = df[column].replace('L A de Herrera', 'AV  DR. LUIS ALBERTO DE HERRERA')
        df[column] = df[column].replace('Benito Blanco', 'JUAN BENITO BLANCO')
        df[column] = df[column].replace('18 de Julio', 'AV 18 DE JULIO')
        df[column] = df[column].replace('Evaristo Ciganda', 'DR EVARISTO CIGANDA')
        df[column] = df[column].replace('Gral Caraballo', 'GRAL FRANCISCO CARABALLO')
        df[column] = df[column].replace('C M Ramirez', 'AV CARLOS MARIA RAMIREZ')
        df[column] = df[column].replace('Paiva', 'BENIGNO PAIVA')
        df[column] = df[column].replace('Varela S', 'AV JOSE PEDRO VARELA')
        df[column] = df[column].replace('Requena y Garcia', 'AV JOSE G REQUENA GARCIA')
        df[column] = df[column].replace('Bonavita', 'DR LUIS BONAVITA')
        df[column] = df[column].replace('Cibils', 'JAIME CIBILS')
        df[column] = df[column].replace('Francisco Simon', 'DR FRANCISCO SIMON')
        df[column] = df[column].replace('Merola', 'AV DR LORENZO MEROLA')
        df[column] = df[column].replace('Serrato','ING JOSE SERRATO')
        df[column] = df[column].replace('Errazquin','MANUEL J ERRAZQUIN')
        df[column] = df[column].replace('Tezanos','TOMAS DE TEZANOS')
        df[column] = df[column].replace('Solano Antuna','FRANCISCO SOLANO ANTUÑA')
        df[column] = df[column].replace('Jackson','JUAN D JACKSON')
        df[column] = df[column].replace('Casavalle','CNO CASAVALLE')
        df[column] = df[column].replace('Penco','DR JOSE MARIA PENCO')
        df[column] = df[column].replace('Ferrer Serra','DR SALVADOR FERRER SERRA')
        df[column] = df[column].replace('Las Heras','GRAL LAS HERAS')
        df[column] = df[column].replace('Gallinal','DR ALEJANDRO GALLINAL')
        df[column] = df[column].replace('Canning','JORGE CANNING')
        df[column] = df[column].replace('Rondeau','AV GRAL RONDEAU')
        df[column] = df[column].replace('Eduardo Raiz','EDUARDO A RAIZ')
        df[column] = df[column].replace('Buxareo','FELIX BUXAREO')
        df[column] = df[column].replace('Rosell y Rius','ALEJO ROSELL Y RIUS')
        df[column] = df[column].replace('Cabrera','JOSE ANTONIO CABRERA')
        df[column] = df[column].replace('Duran','CNO JOSE DURAN')
        df[column] = df[column].replace('Salari','TTE CNEL EUCLIDES A SALARI')
        df[column] = df[column].replace('Canaro','FRANCISCO CANARO')
        df[column] = df[column].replace('Montero','JOSE MARIA MONTERO')
        df[column] = df[column].replace('Sanguinetti','FELIPE SANGUINETTI')
        df[column] = df[column].replace('Freire','JOSE B FREIRE')
        df[column] = df[column].replace('Presidente Berro','PTE BERRO')    
        df[column] = df[column].replace('Gestido','PTE GRAL OSCAR DIEGO GESTIDO')
        df[column] = df[column].replace('Aquiles Lanza','DR AQUILES R LANZA')
        df[column] = df[column].replace('Ariel','CNO ARIEL')
        df[column] = df[column].replace('Terra','JOSE L TERRA')
        df[column] = df[column].replace('Rousseau','JUAN JACOBO ROUSSEAU')
        df[column] = df[column].replace('Gestido','PTE GRAL OSCAR DIEGO GESTIDO')
        df[column] = df[column].replace('18 de julio','AV 18 DE JULIO')
        df[column] = df[column].replace('Llupes','JOSE LLUPES')
        df[column] = df[column].replace('Lucas Obes','LUCAS J OBES')
        df[column] = df[column].replace('Canstatt','ING EDUARDO CANSTATT')
        df[column] = df[column].replace('Cnel Raiz','CNO CORONEL RAIZ')
        df[column] = df[column].replace('Urquiza','GRAL URQUIZA')  
        df[column] = df[column].replace('Tomkinson','CNO TOMKINSON')
        df[column] = df[column].replace('Colombes','AV COLOMBES')
        df[column] = df[column].replace('Sainz Rosas','FRANCISCO SAINZ ROSAS')
        df[column] = df[column].replace('Ramon Anador','AV RAMON ANADOR')
        df[column] = df[column].replace('J P Varela','AV JOSE PEDRO VARELA')
        df[column] = df[column].replace('Quesada','JUAN JOSE QUESADA')
        df[column] = df[column].replace('Brito del Pino','GRAL JOSE ESTEBAN BRITO DEL PINO')
        df[column] = df[column].replace('German Barbato','AGRM GERMAN BARBATO')
        df[column] = df[column].replace('Gral Villagran','GRAL JOSE VILLAGRAN')
        df[column] = df[column].replace('Enrique Martinez','GRAL ENRIQUE MARTINEZ')
        df[column] = df[column].replace('Martin C Martinez','DR MARTIN C MARTINEZ')
        df[column] = df[column].replace('San Quintin N','SAN QUINTIN')
        df[column] = df[column].replace('Gutierrez Ruiz','HECTOR GUTIERREZ RUIZ')
        df[column] = df[column].replace('Gral Paz','AV GRAL JOSE MARIA PAZ')
        df[column] = df[column].replace('Alfredo Navarro','AV DR ALFREDO NAVARRO')
        df[column] = df[column].replace('Sosa','AV JULIO MARIA SOSA')
        df[column] = df[column].replace('Gral San Martin','AV GRAL SAN MARTIN')
        df[column] = df[column].replace('Rep Dominicana','REPUBLICA DOMINICANA')
        df[column] = df[column].replace('Ruta 1','RUTA NAL 1')
        df[column] = df[column].replace('Pereira','GABRIEL A PEREIRA')
        df[column] = df[column].replace('Accesos','RUTA NAL ACCESO RUTA 1 Y 5')
    # Arreglo las camaras que estan situadas en la rambla
    # No puedo hacerlo en loop porque no es inyectiva



    # Arreglo aquellos nombres donde se menciona la calle "Rambla" con su nombre correspondiente

    df.loc[(df['latitud']==-34.915369)&(df['longitud']==-56.168358)&(df['dsc_int_siguiente']=='Rambla'),'dsc_int_siguiente'] = 'RBLA WILSON'
    df.loc[(df['latitud']==-34.897400)&(df['longitud']==-56.120289)&(df['dsc_int_siguiente']=='Rambla'),'dsc_int_siguiente'] = 'RBLA REPUBLICA DE CHILE'
    df.loc[(df['latitud']==-34.924558000000005)&(df['longitud']==-56.155621999999994)&(df['dsc_int_siguiente']=='Rambla'),'dsc_int_siguiente'] = 'RBLA MAHATMA GANDHI' # Este sorete tiene hasta esos puntos decimales de latitud y longitud
    df.loc[(df['latitud']==-34.909636)&(df['longitud']==-56.135381)&(df['dsc_int_siguiente']=='Rambla'),'dsc_int_siguiente'] = 'RBLA ARMENIA'
    df.loc[(df['latitud']==-34.909636)&(df['longitud']==-56.135381)&(df['dsc_int_siguiente']=='Rambla'),'dsc_int_anterior'] = 'RBLA REPUBLICA DEL PERU'
    df.loc[(df['latitud']==-34.909636)&(df['longitud']==-56.135381)&(df['dsc_int_siguiente']=='Rambla'),'dsc_int_siguiente'] = 'RBLA ARMENIA'
    df.loc[(df['latitud']==-34.909983000000004)&(df['longitud']==-56.135456000000005)&(df['dsc_int_siguiente']=='Rambla'),'dsc_int_siguiente'] = 'RBLA REPUBLICA DEL PERU'
    df.loc[(df['dsc_avenida']=='AV  DR. LUIS ALBERTO DE HERRERA')&(df['dsc_int_siguiente']=='26 de Marzo')&(df['dsc_int_anterior']=='Rambla'),'dsc_int_anterior'] = 'RBLA ARMENIA'
    df.loc[(df['dsc_avenida']=='AV SARMIENTO')&(df['dsc_int_siguiente']=='AV JOSE G REQUENA GARCIA')&(df['dsc_int_anterior']=='Rambla'),'dsc_int_anterior'] = 'RBLA WILSON'
    df.loc[(df['dsc_avenida']=='BV ESPAÑA')&(df['dsc_int_siguiente']=='JUAN BENITO BLANCO')&(df['dsc_int_anterior']=='Rambla'),'dsc_int_anterior'] = 'RBLA REPUBLICA DEL PERU'
    df.loc[(df['dsc_avenida']=='26 de Marzo')&(df['dsc_int_siguiente']=='AV  DR. LUIS ALBERTO DE HERRERA')&(df['dsc_int_anterior']=='Rambla'),'dsc_int_anterior'] = 'RBLA ARMENIA'
    df.loc[(df['dsc_int_anterior']=='JOSE MARIA MONTERO')&(df['dsc_int_siguiente']=='BV ESPAÑA')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA REPUBLICA DEL PERU'
    df.loc[(df['dsc_int_anterior']=='Morales')&(df['dsc_int_siguiente']=='JUAN D JACKSON')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA REPUBLICA ARGENTINA'
    df.loc[(df['dsc_int_anterior']=='Arrascaeta')&(df['dsc_int_siguiente']=='Hipolito Yrigoyen')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA REPUBLICA DE CHILE'
    df.loc[(df['dsc_int_anterior']=='AV  DR. LUIS ALBERTO DE HERRERA')&(df['dsc_int_siguiente']=='FELIX BUXAREO')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA REPUBLICA DEL PERU'
    df.loc[(df['dsc_int_anterior']=='AV  DR. LUIS ALBERTO DE HERRERA')&(df['dsc_int_siguiente']=='26 de Marzo')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA ARMENIA'
    df.loc[(df['dsc_int_anterior']=='GRAL DE DIVISION ROBERTO P RIVEROS')&(df['dsc_int_siguiente']=='BENIGNO PAIVA')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA ARMENIA'
    df.loc[(df['dsc_int_anterior']=='AV GRAL JOSE MARIA PAZ')&(df['dsc_int_siguiente']=='DR ALEJANDRO GALLINAL')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA O´HIGGINS'
    df.loc[(df['dsc_int_anterior']=='Hipolito Yrigoyen')&(df['dsc_int_siguiente']=='9 de Junio')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA REPUBLICA DE CHILE'
    df.loc[(df['dsc_int_anterior']=='JOSE MARIA MONTERO')&(df['dsc_int_siguiente']=='Solano Garcia')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA MAHATMA GANDHI'
    df.loc[(df['dsc_int_anterior']=='MANUEL J ERRAZQUIN')&(df['dsc_int_siguiente']=='AV SARMIENTO')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA WILSON'
    df.loc[(df['dsc_int_anterior']=='BV JOSE BATLLE Y ORDOÑEZ')&(df['dsc_int_siguiente']=='AV MCAL FRANCISCO SOLANO LOPEZ')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA REPUBLICA DE CHILE'
    df.loc[(df['dsc_int_anterior']=='BV ESPAÑA')&(df['dsc_int_siguiente']=='JOSE MARIA MONTERO')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA MAHATMA GANDHI'
    df.loc[(df['dsc_int_anterior']=='AV SARMIENTO')&(df['dsc_int_siguiente']=='MANUEL J ERRAZQUIN')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA MAHATMA GANDHI'
    df.loc[(df['dsc_int_anterior']=='JOSE MARIA MONTERO')&(df['dsc_int_siguiente']=='FRANCISCO SOLANO ANTUÑA')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA MAHATMA GANDHI'
    df.loc[(df['dsc_int_anterior']=='JOSE MARIA MONTERO')&(df['dsc_int_siguiente']=='BV ESPAÑA')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA REPUBLICA DEL PERU'
    df.loc[(df['dsc_int_anterior']=='Morales')&(df['dsc_int_siguiente']=='JUAN D JACKSON')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA REPUBLICA ARGENTINA'
    df.loc[(df['dsc_int_anterior']=='Arrascaeta')&(df['dsc_int_siguiente']=='Hipolito Yrigoyen')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA REPUBLICA DE CHILE'
    df.loc[(df['dsc_int_anterior']=='AV  DR. LUIS ALBERTO DE HERRERA')&(df['dsc_int_siguiente']=='FELIX BUXAREO')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA REPUBLICA DEL PERU'
    df.loc[(df['dsc_int_anterior']=='AV  DR. LUIS ALBERTO DE HERRERA')&(df['dsc_int_siguiente']=='26 de Marzo')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA ARMENIA'
    df.loc[(df['dsc_int_anterior']=='GRAL DE DIVISION ROBERTO P RIVEROS')&(df['dsc_int_siguiente']=='BENIGNO PAIVA')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA ARMENIA'
    df.loc[(df['dsc_int_anterior']=='AV GRAL JOSE MARIA PAZ')&(df['dsc_int_siguiente']=='DR ALEJANDRO GALLINAL')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA O´HIGGINS'
    df.loc[(df['dsc_int_anterior']=='Hipolito Yrigoyen')&(df['dsc_int_siguiente']=='9 de Junio')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA REPUBLICA DE CHILE'
    df.loc[(df['dsc_int_anterior']=='JOSE MARIA MONTERO')&(df['dsc_int_siguiente']=='Solano Garcia')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA MAHATMA GANDHI'
    df.loc[(df['dsc_int_anterior']=='MANUEL J ERRAZQUIN')&(df['dsc_int_siguiente']=='AV SARMIENTO')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] ='RBLA WILSON'
    df.loc[(df['dsc_int_anterior']=='BV JOSE BATLLE Y ORDOÑEZ')&(df['dsc_int_siguiente']=='AV MCAL FRANCISCO SOLANO LOPEZ')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA REPUBLICA DE CHILE'
    df.loc[(df['dsc_int_anterior']=='BV ESPAÑA')&(df['dsc_int_siguiente']=='JOSE MARIA MONTERO')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA MAHATMA GANDHI'
    df.loc[(df['dsc_int_anterior']=='AV SARMIENTO')&(df['dsc_int_siguiente']=='MANUEL J ERRAZQUIN')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA MAHATMA GANDHI'
    df.loc[(df['dsc_int_anterior']=='JOSE MARIA MONTERO')&(df['dsc_int_siguiente']=='FRANCISCO SOLANO ANTUÑA')&(df['dsc_avenida']=='Rambla'),'dsc_avenida'] = 'RBLA MAHATMA GANDHI'
    df.loc[(df['dsc_avenida']=='AV  DR. LUIS ALBERTO DE HERRERA')&(df['dsc_int_siguiente']=='RBLA ARMENIA')&(df['dsc_int_anterior']=='Rambla'),'dsc_int_anterior'] = 'RBLA REPUBLICA DEL PERU'
    df.loc[(df['dsc_avenida']=='Rambla')&(df['dsc_int_anterior']=='GRAL DE DIVISION ROBERTO P RIVEROS')&(df['dsc_int_siguiente']=='BENIGNO PAIVA'),'dsc_avenida'] = 'RBLA ARMENIA'

    # Desambiguo calles que tienen el mismo nombre en las camaras pero son calles diferentes. 
    df.loc[(df['dsc_int_siguiente']=='BENIGNO PAIVA')&(df['dsc_int_anterior']=='Pereyra'),'dsc_int_anterior'] = 'GRAL DE DIVISION ROBERTO P RIVEROS'
    df.loc[(df['dsc_int_siguiente']=='AV GRAL RIVERA')&(df['dsc_int_anterior']=='Pereyra'),'dsc_int_anterior'] = 'AV ING JUANA PEREYRA'
    
    return df

#df = fix_street_labels(df)

#df.drop_duplicates(keep='first',inplace=True)



##### Crea el archivo con la posicion de las camaras

In [17]:
df.to_csv('Posicion de camaras.csv',index=False)

##### Genera la base de datos con las calles arregladas

In [18]:
def create_database(id_dict,path=r'C:\Python\Master Datascience\Tesis\dataset'):
    """Arregla las calles y las pasa a una base de datos"""
    #engine = create_engine('sqlite://', echo=False)
    con = sqlite3.connect('conteo_vehicular_id_corregido.sqlite')
    
    print('Conectado a la base')
    delete = 'DROP TABLE data'
    
    cur = con.cursor()
    try:
        cur.execute(delete)
        con.commit()
        print("Borrado con exito")
    except Exception as e:
        print(e)
    all_files = glob.glob(path + "/*.csv")
    counter = 0
    chunksize = 10 ** 5
    for filename in all_files:
        #print("Van {}, faltan {}".format(counter,len(all_files)-counter))
        try:
            for chunk in pd.read_csv(filename ,header=0 ,encoding="ISO-8859-1",chunksize=chunksize, error_bad_lines=False):
                chunk = chunk.rename(columns={chunk.columns[0]:'id_camara'})
                chunk.drop_duplicates(keep='first',inplace=True)
                chunk = fix_street_labels(chunk)
                chunk['mix'] = chunk.dsc_avenida + '/' + chunk.dsc_int_anterior + '/' +  chunk.dsc_int_siguiente
                chunk['id_corregido'] = chunk.mix.map(id_dict)
                chunk['mixdate'] = chunk.fecha + ' ' + chunk.hora
                chunk['mixdate']= pd.to_datetime(chunk.mixdate,format='%Y-%m-%d %H:%M:%S.%f')
                chunk.drop(['fecha','hora'],axis=1,inplace=True)
                chunk.rename(columns={"mixdate": 'Fecha_Hora'},inplace=True)
                chunk.to_sql('data', con=con, if_exists='append',index=False)
        except Exception as e:
            print("ERROR",e,filename)
            continue
        counter += 1
        print('van ',counter)
        print(filename)
    del chunk
    faltan = ([r'C:\Python\Master Datascience\Tesis\dataset\autoscope_05_2020.csv',
           r'C:\Python\Master Datascience\Tesis\dataset\autoscope_06_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_07_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_08_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\Autoscope_09_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_10_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_11_2020.csv',
          r'C:\Python\Master Datascience\Tesis\dataset\autoscope_12_2020.csv'])
    
    for filename in faltan:
        print("Va el file", filename)
        for chunk in pd.read_csv(filename ,header=0 ,encoding="ISO-8859-1",sep=";",chunksize=chunksize, error_bad_lines=False):
            chunk = chunk.rename(columns={chunk.columns
                                          [0]:'id_camara','volume':'volumen'})
            chunk.drop_duplicates(keep='first',inplace=True)
            chunk = fix_street_labels(chunk)
            chunk['mix'] = chunk.dsc_avenida + '/' + chunk.dsc_int_anterior + '/' +  chunk.dsc_int_siguiente
            chunk['id_corregido'] = chunk.mix.map(id_dict)
            chunk['mixdate'] = chunk.fecha + ' ' + chunk.hora
            chunk['mixdate']= pd.to_datetime(chunk.mixdate,format='%Y-%m-%d %H:%M:%S.%f')
            chunk.drop(['fecha','hora'],axis=1,inplace=True)
            chunk.rename(columns={"mixdate": 'Fecha_Hora'},inplace=True)
            chunk.to_sql('data', con=con, if_exists='append',index=False)

    con.commit()
with open('id_calles_norm.json') as d:
    id_calles_norm = json.load(d)
create_database(id_calles_norm)
print('Exito')

Conectado a la base
Borrado con exito
van  1
C:\Python\Master Datascience\Tesis\dataset\autoscope_01_2020.csv
van  2
C:\Python\Master Datascience\Tesis\dataset\autoscope_02_2020.csv
van  3
C:\Python\Master Datascience\Tesis\dataset\autoscope_03_2020.csv
van  4
C:\Python\Master Datascience\Tesis\dataset\autoscope_04_2020.csv
van  5
C:\Python\Master Datascience\Tesis\dataset\Autoscope_05_2019.csv
ERROR 'dsc_avenida' C:\Python\Master Datascience\Tesis\dataset\autoscope_05_2020.csv
van  6
C:\Python\Master Datascience\Tesis\dataset\Autoscope_06_2019.csv
ERROR 'dsc_avenida' C:\Python\Master Datascience\Tesis\dataset\autoscope_06_2020.csv
van  7
C:\Python\Master Datascience\Tesis\dataset\Autoscope_07_2019.csv
ERROR 'dsc_avenida' C:\Python\Master Datascience\Tesis\dataset\Autoscope_07_2020.csv
van  8
C:\Python\Master Datascience\Tesis\dataset\Autoscope_08_2019.csv
ERROR 'dsc_avenida' C:\Python\Master Datascience\Tesis\dataset\Autoscope_08_2020.csv
van  9
C:\Python\Master Datascience\Tesis\data

In [23]:
def add_MTOP_to_database():
    """Adds all MTOP dataframes to sqlite3 database with IMM Data"""
    #engine = create_engine('sqlite://', echo=False)
    all_files = (['MTOP_2017_formato_IMM.csv','MTOP_2018_formato_IMM.csv',
                  'MTOP_2019_formato_IMM.csv','MTOP_2020_formato_IMM.csv'])
    con = sqlite3.connect('conteo_vehicular_id_corregido.sqlite')
    counter = 0
    for filename in all_files:
        print("Van {}, faltan {}".format(counter,len(all_files)-counter))
        df = pd.read_csv(filename, index_col=None, header=0, encoding="ISO-8859-1")
        df = adapt_MTOP_to_database(df)
        df.to_sql('data', con=con, if_exists='append',index=False)
        counter += 1
    con.commit()
add_MTOP_to_database()
print('Exito')

Van 0, faltan 4
Index(['Fecha_Hora', 'volumen', 'volumen_hora', 'latitud', 'longitud',
       'id_camara', 'id_carril', 'dsc_avenida', 'dsc_int_anterior',
       'dsc_int_siguiente', 'id_corregido'],
      dtype='object')
Van 1, faltan 3
Index(['Fecha_Hora', 'volumen', 'volumen_hora', 'latitud', 'longitud',
       'id_camara', 'id_carril', 'dsc_avenida', 'dsc_int_anterior',
       'dsc_int_siguiente', 'id_corregido'],
      dtype='object')
Van 2, faltan 2
Index(['Fecha_Hora', 'volumen', 'volumen_hora', 'latitud', 'longitud',
       'id_camara', 'id_carril', 'dsc_avenida', 'dsc_int_anterior',
       'dsc_int_siguiente', 'id_corregido'],
      dtype='object')
Van 3, faltan 1
Index(['Fecha_Hora', 'volumen', 'volumen_hora', 'latitud', 'longitud',
       'id_camara', 'id_carril', 'dsc_avenida', 'dsc_int_anterior',
       'dsc_int_siguiente', 'id_corregido'],
      dtype='object')
Exito


In [25]:
conn = sqlite3.connect('conteo_vehicular_id_corregido.sqlite')
query = "SELECT * FROM data WHERE latitud = '-34.864379' ORDER BY ROWID ASC LIMIT 100  "
df = pd.read_sql_query(query, conn)
df.head()

,id_camara,id_carril,dsc_avenida,dsc_int_anterior,dsc_int_siguiente,latitud,longitud,volumen,volumen_hora,mix,id_corregido,Fecha_Hora
0,999,999,RUTA NAL INTERCONEXION RUTA 1 Y 5,AV DR SANTIN CARLOS ROSSI,BV GRAL ARTIGAS,-34.864379,-56.252485,7,7,None,276,2017-01-01 00:00:00
1,999,999,RUTA NAL INTERCONEXION RUTA 1 Y 5,AV DR SANTIN CARLOS ROSSI,BV GRAL ARTIGAS,-34.864379,-56.252485,11,18,None,276,2017-01-01 00:05:00
2,999,999,RUTA NAL INTERCONEXION RUTA 1 Y 5,AV DR SANTIN CARLOS ROSSI,BV GRAL ARTIGAS,-34.864379,-56.252485,14,32,None,276,2017-01-01 00:10:00
3,999,999,RUTA NAL INTERCONEXION RUTA 1 Y 5,AV DR SANTIN CARLOS ROSSI,BV GRAL ARTIGAS,-34.864379,-56.252485,13,45,None,276,2017-01-01 00:15:00
4,999,999,RUTA NAL INTERCONEXION RUTA 1 Y 5,AV DR SANTIN CARLOS ROSSI,BV GRAL ARTIGAS,-34.864379,-56.252485,12,57,None,276,2017-01-01 00:20:00


In [22]:
def create_csvs(id_dict,path=r'C:\Python\Master Datascience\Tesis\dataset'):
    """Arregla las calles y los pasa a csv corregidos"""
    all_files = glob.glob(path + "/*.csv")
    destination_path = './csvdata/'
    
    chunksize = 10 ** 6
    for filename in all_files:
        base_name = destination_path + filename.split('\\')[-1].split(".")[0]
        counter = 0
        try:
            for chunk in pd.read_csv(filename ,header=0 ,encoding="ISO-8859-1",chunksize=chunksize, error_bad_lines=False):
                chunk = chunk.rename(columns={chunk.columns[0]:'id_camara'})
                chunk.drop_duplicates(keep='first',inplace=True)
                chunk = fix_street_labels(chunk)
                chunk['mix'] = chunk.dsc_avenida + '/' + chunk.dsc_int_anterior + '/' +  chunk.dsc_int_siguiente
                chunk['id_corregido'] = chunk.mix.map(id_dict)
                name = base_name + "_{}.csv".format(counter) 
                chunk.to_csv(name,index=False)
                counter += 1
                print("{} guardado con exito".format(name))
        except Exception as e:
            print("ERROR",e,filename)
            continue
    del chunk
    faltan = ([r'E:\Python\Master Datascience\Tesis\dataset\autoscope_05_2020.csv',
           r'E:\Python\Master Datascience\Tesis\dataset\autoscope_06_2020.csv',
          r'E:\Python\Master Datascience\Tesis\dataset\Autoscope_07_2020.csv',
          r'E:\Python\Master Datascience\Tesis\dataset\Autoscope_08_2020.csv',
          r'E:\Python\Master Datascience\Tesis\dataset\Autoscope_09_2020.csv',
          r'E:\Python\Master Datascience\Tesis\dataset\autoscope_10_2020.csv'])
    
    for filename in faltan:
        print("Va el file", filename)
        counter = 0
        base_name = destination_path + filename.split('\\')[-1].split(".")[0]
        for chunk in pd.read_csv(filename ,header=0 ,encoding="ISO-8859-1",sep=";",chunksize=chunksize, error_bad_lines=False):
            chunk = chunk.rename(columns={chunk.columns[0]:'id_camara','volume':'volumen'})
            chunk.drop_duplicates(keep='first',inplace=True)
            chunk = fix_street_labels(chunk)
            chunk['mix'] = chunk.dsc_avenida + '/' + chunk.dsc_int_anterior + '/' +  chunk.dsc_int_siguiente
            chunk['id_corregido'] = chunk.mix.map(id_dict)
            name = base_name + "_{}.csv".format(counter)
            chunk.to_csv(name,index=False)
            counter += 1
            print("{} guardado con exito".format(name))

In [23]:
with open('id_calles_norm.json') as d:
    id_calles_norm = json.load(d)
create_csvs(id_calles_norm)

KeyboardInterrupt: 

In [ ]:
def create_csv(path):
    all_files = glob.glob(path + "/*.csv")
    counter = 0
    for filename in all_files:
        print("Van {}, faltan {}".format(counter,len(all_files)-counter))
        df = pd.read_csv(filename, index_col=None, header=0, encoding="ISO-8859-1")
        df = fix_street_labels(df)
        df.to_sql('data', con=con, if_exists='append',index=False)
    con.commit()
print('Exito')

##### Reveer esto a la luz de los arreglos en el nombre de las calles

El total de camaras es de 301 camaras coinciden en latitud y longitud con otra. Existen 15 pares que o estan realmente en la misma posicion pero apuntando a diferentes calles oestan cerca. como , una en Rivera entre Artigas y Ponce, la otra en Rodo (cruzando Artigas).  El resto tiene todas las mismas coordenadas (-34.902829, -56.164944) de una posicion en el prado pero están etiquetadas como diferentes calles en la ciudad.

In [ ]:
misma_posicion = df.groupby("latitud").filter(lambda x: len(x) == 2)
misma_posicion.sort_values('latitud')
len(misma_posicion)

#### Veamos camaras incoherentes (aparecen en latitudes y longitudes que no deberian) 

In [ ]:
incoherentes = df.groupby("latitud").filter(lambda x: len(x) > 2).drop_duplicates(subset=['dsc_avenida','dsc_int_anterior','dsc_int_siguiente','latitud','longitud'],keep='first')
print("Hay ",len(incoherentes)," incoherentes")
incoherentes.sort_values('latitud')
incoherentes[incoherentes['longitud']==-56.160908]

Hay camaras quee tienen la misma latitud y longitud pero apuntan a lugares distintos. Ese lugares la plaza rodney en el cerro

Crea funciones para comenzar a obtener informacion de los datasets

In [ ]:
def open_frames_for_observations(path=path_test):
    all_files = glob.glob(path + "/*.csv")
    cols = ['longitud','latitud','fecha','hora','volumen_hora','dsc_avenida'] #'dsc_int_siguiente','dsc_int_anterior'
    
    #opened_files = 0
    for i in range(2):
    #for filename in all_files:
        #print(i)
        try:
            df = pd.read_csv(all_files[i], index_col=None, header=0, encoding="ISO-8859-1",usecols=cols)
            df['mix'] =  df.fecha + ' ' + df.hora
            df.mix = pd.to_datetime(df.mix)
            df.drop(['fecha','hora'],axis=1,inplace=True)
            df.rename(columns={"mix": 'fecha'},inplace=True)
            #li.append(df)
            df['file'] = str(filename)
        except:
            continue
    return df

In [ ]:
def evaluate_total_volumen_day(path=path_test):
    """Evalua los volumnes de trafico diarios de las camaras"""
    all_files = glob.glob(path + "/*.csv")
    cols = ['fecha','volumen_hora','dsc_avenida','dsc_int_siguiente','dsc_int_anterior'] 
    li = []
    for filename in all_files:
        try:
            df = pd.read_csv(filename, index_col=None, header=0, encoding="ISO-8859-1",usecols=cols)
            by_day = df.groupby(['fecha','dsc_avenida','dsc_int_siguiente','dsc_int_anterior'] ).sum()
            li.append(by_day)
        except:
            None
    return pd.concat(li, axis=0, ignore_index=False)

In [ ]:
by_day = evaluate_total_volumen_day(path_all)
by_day.rename(columns={'volumen_hora':'volumen_dia'},inplace=True)
by_day.reset_index(inplace=True)
#by_day.drop(['level_0','index'],axis=1,inplace=True) #Borro las columnas generadas por la concat
by_day.drop_duplicates(subset=by_day.columns,keep='first',inplace=True)
by_day.to_csv('Volumen diario todas las camaras.csv')

Abro la base de datos de cruces para ver si las esquinas que estan etiquetadas las camaras son coherentes. 

In [ ]:
cnx = sqlite3.connect('cruces_montevideo.sqlite')
cruces = pd.read_sql_query("SELECT * FROM cruces_montevideo", cnx)
cruces.drop(columns=['gid','cod_depto','ogc_fid','cod_nombre','cod_nom_01'])
df.loc[0]['dsc_avenida'].upper() in cruces.nom_calle_.unique()
cruces.nom_calle_.unique()# La 'Rambla' aparece todo con el mismo nombre en el df de las camaras. lo voy a dividir segun google maps.
not_in = [x for x in df.dsc_avenida.unique() if x.upper() not in cruces.nom_calle_.unique()]